In [3]:
from glob import glob
import sentencepiece as spm
from tokenizers import SentencePieceUnigramTokenizer, Tokenizer

text_sample = """We aimed to evaluate the effect of sleep quality on memory, executive function, and language performance in patients with refractory focal epilepsy and controlled epilepsy and compare these
 with healthy individuals. We prospectively enrolled 37 adolescent and adult patients with refractory focal epilepsy.

How to avoid anti-clockwise rotation animation when reseting rotation from 360deg to 0 deg?

I am creating an animation that looks like a fancy wheel, When resetting rotation from 360deg to 0 deg, It animating the wheel in anti-clockwise direction, How to Avoid this???
HTML
<ul class="cm">
  <li><span>01</span></li>
  <li><span>02</span></li>
  <li><span>03</span></li>
  <li><span>04</span></li>
  <li><span>05</span></li>
  <li><span>06</span></li>
  <li><span>07</span></li>
  <li><span>08</span></li>

</ul>
 
"""

print("Split on whitespace", len(text_sample.split()), "tokens")

for t in (4, 8, 16, 32):
    print("-"*100)
#     print(f"Loading {t}k_model")
#     tokenizer = Tokenizer.from_file(f"tokenizers/pile_{t}.json")
#     tokenized = tokenizer.encode(text_sample)
#     print(len(tokenized), "tokens")
#     print("-" *100)
#     print(" ".join(tokenized.tokens))
    
    sp = spm.SentencePieceProcessor(model_file=f'spmodels/pile_{t}k.model', add_eos=True)
    
    tokenized = sp.encode(["some text", "somte"])
#     print(text_sample.splitlines())
    print(len(tokenized), "tokens")
    print("-" *100)
#     print(list(map(" ".join, tokenized)))
    print(tokenized)

# print("-" *100)
# print("Albert-tokenizer")
# tok = AlbertTokenizer.from_pretrained("albert-large-v2")
# altok = tok.encode(text_sample)
# print(len(altok), "tokens")
# print("-" *100)
# print(" ".join(tok.convert_ids_to_tokens(altok)))

Split on whitespace 99 tokens
----------------------------------------------------------------------------------------------------
2 tokens
----------------------------------------------------------------------------------------------------
[[292, 1924, 2], [158, 40, 232, 2]]
----------------------------------------------------------------------------------------------------
2 tokens
----------------------------------------------------------------------------------------------------
[[233, 1431, 2], [137, 52, 400, 2]]
----------------------------------------------------------------------------------------------------
2 tokens
----------------------------------------------------------------------------------------------------
[[192, 1163, 2], [4978, 452, 2]]
----------------------------------------------------------------------------------------------------
2 tokens
----------------------------------------------------------------------------------------------------
[[173, 1111, 2], [390

In [6]:
"""Save tokenizers into json"""

from tokenizers import SentencePieceUnigramTokenizer

for t in (4, 8, 16, 32):
    print("-"*100)
    print(f"Loading {t}k_model")
    tokenizer = SentencePieceUnigramTokenizer.from_spm(f"spmodels/pile_{t}k.model")
    tokenizer.save(f"tokenizers/pile_{t}.json", pretty=True)

----------------------------------------------------------------------------------------------------
Loading 4k_model
----------------------------------------------------------------------------------------------------
Loading 8k_model
----------------------------------------------------------------------------------------------------
Loading 16k_model
----------------------------------------------------------------------------------------------------
Loading 32k_model


In [11]:
import time
import sentencepiece as spm
from tokenizers import Tokenizer

text_sample = open("../data/enwik8/train.txt.raw").read().splitlines()

for t in (4, 8, 16, 32):
    print("-"*100)
    print(f"Loading tokenizers {t}k_model")
    tokenizer = Tokenizer.from_file(f"tokenizers/pile_{t}.json")
    
    starttime = time.time()
    tokenized = tokenizer.encode_batch(text_sample, add_special_tokens=True)
    
    print(sum(len(x) for x in tokenized), "tokens")
    print("processed in ", int(time.time() - starttime), "secs")
    print("-" *100)
    
for t in (4, 8, 16, 32):
    print("-"*100)
    print(f"Loading spm {t}k_model")
    sp = spm.SentencePieceProcessor(model_file=f'spmodels/pile_{t}k.model')

    starttime = time.time()
    sptokenized = sp.encode(text_sample)
    
    print(sum(len(x) for x in sptokenized), "tokens")
    print("processed in ", int(time.time() - starttime), "secs")
    print("-" *100)

----------------------------------------------------------------------------------------------------
Loading tokenizers 4k_model
34051862 tokens
processed in  17 secs
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Loading tokenizers 8k_model
30989028 tokens
processed in  20 secs
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Loading tokenizers 16k_model
26695247 tokens
processed in  19 secs
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Loading tokenizers 32k_model
24625391 tokens
processed in  18 secs
---------------------------

In [5]:
"""Save tokenizers into json"""

import time
import sentencepiece as spm
from tokenizers import Tokenizer

text_sample = open("../data/enwik8/train.txt.raw").read().splitlines()

print("-"*100)
print(f"Loading spm 16k_model")
sp = spm.SentencePieceProcessor(model_file=f'spmodels/pile_16k.model')

starttime = time.time()
sptokenized = sp.encode(text_sample)

print("processed in ", int(time.time() - starttime), "secs")
print("-" *100)

----------------------------------------------------------------------------------------------------
Loading spm 16k_model
processed in  23 secs
----------------------------------------------------------------------------------------------------


In [12]:
import itertools

al = 

In [13]:
len(al)

26695247